# Deep Q-Learning 

For this assignment we will implement the Deep Q-Learning algorithm with Experience Replay as described in breakthrough paper __"Playing Atari with Deep Reinforcement Learning"__. We will train an agent to play the famous game of __Breakout__.

In [1]:
import sys
import gym
import torch
import pylab
import random
import numpy as np
import time
from collections import deque
from datetime import datetime
from copy import deepcopy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from utils import *
from agent import *
from model import *
from config import *
from env import GameEnv
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Understanding the environment

In the following cell, we initialise our game of __Breakout__ and you can see how the environment looks like. For further documentation of the of the environment refer to https://gym.openai.com/envs. 

In [2]:
envs = []
for i in range(num_envs):
    envs.append(GameEnv('SpaceInvadersDeterministic-v4'))
#env.render()

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [3]:
number_lives = envs[0].life
state_size = envs[0].observation_space.shape
action_size = envs[0].action_space.n
rewards, episodes = [], []

## Creating a DQN Agent

Here we create a DQN Agent. This agent is defined in the __agent.py__. The corresponding neural network is defined in the __model.py__. 

__Evaluation Reward__ : The average reward received in the past 100 episodes/games.

__Frame__ : Number of frames processed in total.

__Memory Size__ : The current size of the replay memory.

In [4]:
agent = Agent(action_size)
torch.save(agent.policy_net.state_dict(), "./save_model/spaceinvaders_ppo_best")
evaluation_reward = deque(maxlen=evaluation_reward_length)
frame = 0
memory_size = 0
reset_max = 10

### Main Training Loop

In [2]:
vis_env_idx = 0
vis_env = envs[vis_env_idx]
e = 0
frame = 0
max_eval = -np.inf
reset_count = 0

while (frame < 10000000):
    step = 0
    assert(num_envs * env_mem_size == train_frame)
    frame_next_vals = []
    for i in range(num_envs):
        env = envs[i]
        #history = env.history
        #life = env.life
        #state, reward, done, info = [env.state, env.reward, env.done, env.info]
        for j in range(env_mem_size):
            step += 1
            frame += 1
            
            curr_state = env.history[HISTORY_SIZE-1,:,:]
            action, value = agent.get_action(np.float32(env.history[:HISTORY_SIZE,:,:]) / 255.)
            
            next_state, env.reward, env.done, env.info = env.step(action)
            
            if (i == vis_env_idx):
                vis_env._env.render()
            
            frame_next_state = get_frame(next_state)
            env.history[HISTORY_SIZE,:,:] = frame_next_state
            terminal_state = check_live(env.life, env.info['ale.lives'])
            
            env.life = env.info['ale.lives']
            r = env.reward
            
            agent.memory.push(i, deepcopy(curr_state), action, r, terminal_state, value, 0, 0)
            if (j == env_mem_size-1):
                _, frame_next_val = agent.get_action(np.float32(env.history[1:,:,:]) / 255.)
                frame_next_vals.append(frame_next_val)
            env.score += r
            env.history[:HISTORY_SIZE, :, :] = env.history[1:,:,:]
            
            if (env.done):
                if (e % 50 == 0):
                    print('now time : ', datetime.now())
                    rewards.append(np.mean(evaluation_reward))
                    episodes.append(e)
                    pylab.plot(episodes, rewards, 'b')
                    pylab.savefig("./save_graph/spaceinvaders_ppo.png")
                    torch.save(agent.policy_net, "./save_model/spaceinvaders_ppo")
                    
                    if np.mean(evaluation_reward) > max_eval:
                        torch.save(agent.policy_net.state_dict(), "./save_model/spaceinvaders_ppo_best")
                        max_eval = float(np.mean(evaluation_reward))
                        reset_count = 0
                    elif e > 5000:
                        reset_count += 1
                        """
                        if (reset_count == reset_max):
                            print("Training went nowhere, starting again at best model")
                            agent.policy_net.load_state_dict(torch.load("./save_model/spaceinvaders_ppo_best"))
                            agent.update_target_net()
                            reset_count = 0
                        """
                e += 1
                evaluation_reward.append(env.score)
                print("episode:", e, "  score:", env.score,  " epsilon:", agent.epsilon, "   steps:", step,
                  " evaluation reward:", np.mean(evaluation_reward))
                
                env.done = False
                env.score = 0
                env.history = np.zeros([HISTORY_SIZE+1,84,84], dtype=np.uint8)
                env.state = env.reset()
                env.life = number_lives
                get_init_state(env.history, env.state)
                
                
                
    agent.train_policy_net(frame, frame_next_vals)
    agent.update_target_net()


NameError: name 'envs' is not defined

In [ ]:
torch.save(agent.policy_net, "./save_model/spaceinvaders_ppo")

In [ ]:
### Loop through all environments and run PPO on them

env_names = ['SpaceInvaders-v0', 'MsPacman-v0', 'Asteroids-v0', 'Asterix-v0', 'Atlantis-v0', 'Alien-v0', 'Amidar-v0', 'Assault-v0', 'BankHeist-v0']

for a in range(len(env_names)):
    name = env_names[a]
    print("\n\n\n ------- STARTING TRAINING FOR %s ------- \n\n\n" % (name))
    
    envs = []
    for i in range(num_envs):
        envs.append(GameEnv(name))
    #env.render()
    

    number_lives = envs[0].life
    state_size = envs[0].observation_space.shape
    if (name == 'SpaceInvaders-v0'):
        action_size = 4
    else:
        action_size = envs[0].action_space.n
    rewards, episodes = [], []

    vis_env_idx = 0
    vis_env = envs[vis_env_idx]
    e = 0
    frame = 0
    max_eval = -np.inf
    reset_count = 0


    agent = Agent(action_size)
    torch.save(agent.policy_net.state_dict(), "./save_model/" + name + "_best")
    evaluation_reward = deque(maxlen=evaluation_reward_length)
    frame = 0
    memory_size = 0
    reset_max = 10
    
    print("Determing min/max rewards of environment")
    [low, high] = score_range = get_score_range(name)
    print("Min: %d. Max: %d." % (low, high))

    while (frame < 10000000):
        step = 0
        assert(num_envs * env_mem_size == train_frame)
        frame_next_vals = []
        
        for j in range(env_mem_size):
            
            curr_states = np.stack([envs[i].history[HISTORY_SIZE-1,:,:] for i in range(num_envs)])
            next_states = []
            net_in = np.stack([envs[i].history[:HISTORY_SIZE,:,:] for i in range(num_envs)])
            step += num_envs
            frame += num_envs
            actions, values = agent.get_action(np.float32(net_in) / 255.)
            
            for i in range(num_envs):
                env = envs[i]
                next_state, env.reward, env.done, env.info = env.step(actions[i])
                next_states.append(next_state)
                if (i == vis_env_idx):
                    vis_env._env.render()
            
            for i in range(num_envs):
                env = envs[i]
                """
                next_state, env.reward, env.done, env.info = env.step(actions[i])
                if (i == vis_env_idx):
                    vis_env._env.render()
                """
                
                frame_next_state = get_frame(next_states[i])
                env.history[HISTORY_SIZE,:,:] = frame_next_state
                terminal_state = check_live(env.life, env.info['ale.lives'])
                env.life = env.info['ale.lives']
                r = (env.reward / high) * 20.0 #np.log(max(env.reward+1, 1))#((env.reward - low) / (high - low)) * 30
                agent.memory.push(i, deepcopy(curr_states[i]), actions[i], r, terminal_state, values[i], 0, 0)
                
                if (j == env_mem_size-1):
                    net_in = np.stack([envs[k].history[1:,:,:] for k in range(num_envs)])
                    _, frame_next_vals = agent.get_action(np.float32(net_in) / 255.)
                
                env.score += env.reward
                env.history[:HISTORY_SIZE, :, :] = env.history[1:,:,:]
        
                if (env.done):
                    if (e % 50 == 0):
                        print('now time : ', datetime.now())
                        rewards.append(np.mean(evaluation_reward))
                        episodes.append(e)
                        pylab.plot(episodes, rewards, 'b')
                        pylab.savefig("./save_graph/" + name + "_ppo.png")
                        torch.save(agent.policy_net, "./save_model/" + name + "_ppo")

                        if np.mean(evaluation_reward) > max_eval:
                            torch.save(agent.policy_net.state_dict(), "./save_model/"  + name + "_ppo_best")
                            max_eval = float(np.mean(evaluation_reward))
                            reset_count = 0
                        elif e > 5000:
                            reset_count += 1
                            """
                            if (reset_count == reset_max):
                                print("Training went nowhere, starting again at best model")
                                agent.policy_net.load_state_dict(torch.load("./save_model/spaceinvaders_ppo_best"))
                                agent.update_target_net()
                                reset_count = 0
                            """
                    e += 1
                    evaluation_reward.append(env.score)
                    print("episode:", e, "  score:", env.score,  " epsilon:", agent.epsilon, "   steps:", step,
                      " evaluation reward:", np.mean(evaluation_reward))

                    env.done = False
                    env.score = 0
                    env.history = np.zeros([HISTORY_SIZE+1,84,84], dtype=np.uint8)
                    env.state = env.reset()
                    env.life = number_lives
                    get_init_state(env.history, env.state)
            
        agent.train_policy_net(frame, frame_next_vals)
        agent.update_target_net()
    print("FINISHED TRAINING FOR %s" % (name))
    pylab.figure()




 ------- STARTING TRAINING FOR SpaceInvaders-v0 ------- 





/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Determing min/max rewards of environment
Min: 0. Max: 200.


/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/model.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(x[:,:self.action_size] - torch.max(x[:,:self.action_size],1)[0].unsqueeze(1))


Training network. lr: 0.000250. clip: 0.100000


...

Iteration 1335: Policy loss: 0.007622. Value loss: 4.456735. Entropy: 0.408282.
episode: 550   score: 180.0  epsilon: 1.0    steps: 424  evaluation reward: 206.25
Training network. lr: 0.000240. clip: 0.096009
Iteration 1336: Policy loss: 0.007625. Value loss: 5.191298. Entropy: 0.356188.
Iteration 1337: Policy loss: -0.001100. Value loss: 4.368897. Entropy: 0.351298.
Iteration 1338: Policy loss: 0.001329. Value loss: 4.378137. Entropy: 0.341249.
now time :  2019-03-05 16:52:06.419203
episode: 551   score: 210.0  epsilon: 1.0    steps: 984  evaluation reward: 206.25
episode: 552   score: 180.0  epsilon: 1.0    steps: 1008  evaluation reward: 206.25
Training network. lr: 0.000240. clip: 0.096009
Iteration 1339: Policy loss: 0.003849. Value loss: 11.631086. Entropy: 0.384774.
Iteration 1340: Policy loss: 0.004064. Value loss: 8.707512. Entropy: 0.381872.
Iteration 1341: Policy loss: 0.019512. Value loss: 8.767014. Entropy: 0.409127.
episode: 553   score: 180.0  epsilon: 1.0    steps: 160

episode: 576   score: 180.0  epsilon: 1.0    steps: 368  evaluation reward: 203.95
episode: 577   score: 155.0  epsilon: 1.0    steps: 808  evaluation reward: 203.7
Training network. lr: 0.000240. clip: 0.095862
Iteration 1399: Policy loss: 0.006111. Value loss: 13.227863. Entropy: 0.619390.
Iteration 1400: Policy loss: 0.017048. Value loss: 7.813485. Entropy: 0.608069.
Iteration 1401: Policy loss: 0.019759. Value loss: 7.014798. Entropy: 0.619438.
episode: 578   score: 210.0  epsilon: 1.0    steps: 192  evaluation reward: 203.2
Training network. lr: 0.000239. clip: 0.095705
Iteration 1402: Policy loss: 0.004041. Value loss: 6.194780. Entropy: 0.459858.
Iteration 1403: Policy loss: 0.008147. Value loss: 4.102281. Entropy: 0.450624.
Iteration 1404: Policy loss: 0.006496. Value loss: 4.058338. Entropy: 0.468030.
Training network. lr: 0.000239. clip: 0.095705
Iteration 1405: Policy loss: 0.004336. Value loss: 11.262110. Entropy: 0.415684.
Iteration 1406: Policy loss: 0.009344. Value loss:

Iteration 1461: Policy loss: 0.023801. Value loss: 7.872222. Entropy: 0.643977.
Training network. lr: 0.000239. clip: 0.095549
Iteration 1462: Policy loss: 0.007948. Value loss: 6.885470. Entropy: 0.470411.
Iteration 1463: Policy loss: 0.009963. Value loss: 3.687673. Entropy: 0.436145.
Iteration 1464: Policy loss: 0.008817. Value loss: 3.067862. Entropy: 0.427217.
episode: 603   score: 180.0  epsilon: 1.0    steps: 584  evaluation reward: 193.9
episode: 604   score: 155.0  epsilon: 1.0    steps: 816  evaluation reward: 193.65
Training network. lr: 0.000239. clip: 0.095549
Iteration 1465: Policy loss: 0.011358. Value loss: 15.361852. Entropy: 0.555090.
Iteration 1466: Policy loss: 0.005274. Value loss: 8.272793. Entropy: 0.544823.
Iteration 1467: Policy loss: 0.006060. Value loss: 7.026684. Entropy: 0.538951.
episode: 605   score: 105.0  epsilon: 1.0    steps: 360  evaluation reward: 192.9
Training network. lr: 0.000239. clip: 0.095549
Iteration 1468: Policy loss: 0.012736. Value loss: 

Iteration 1523: Policy loss: 0.000512. Value loss: 5.604450. Entropy: 0.479644.
Iteration 1524: Policy loss: 0.006678. Value loss: 4.641637. Entropy: 0.482540.
episode: 630   score: 105.0  epsilon: 1.0    steps: 832  evaluation reward: 184.6
Training network. lr: 0.000239. clip: 0.095401
Iteration 1525: Policy loss: 0.000615. Value loss: 10.101652. Entropy: 0.416528.
Iteration 1526: Policy loss: -0.001112. Value loss: 6.873206. Entropy: 0.444082.
Iteration 1527: Policy loss: -0.000096. Value loss: 5.802977. Entropy: 0.448738.
episode: 631   score: 180.0  epsilon: 1.0    steps: 680  evaluation reward: 184.6
episode: 632   score: 210.0  epsilon: 1.0    steps: 968  evaluation reward: 182.45
Training network. lr: 0.000239. clip: 0.095401
Iteration 1528: Policy loss: 0.005733. Value loss: 8.595915. Entropy: 0.363086.
Iteration 1529: Policy loss: 0.002390. Value loss: 6.275974. Entropy: 0.376746.
Iteration 1530: Policy loss: 0.001315. Value loss: 5.236938. Entropy: 0.377746.
episode: 633   s

Iteration 1587: Policy loss: 0.001876. Value loss: 17.029335. Entropy: 0.314574.
episode: 655   score: 210.0  epsilon: 1.0    steps: 272  evaluation reward: 187.75
Training network. lr: 0.000238. clip: 0.095245
Iteration 1588: Policy loss: 0.002437. Value loss: 18.724773. Entropy: 0.311004.
Iteration 1589: Policy loss: 0.002236. Value loss: 9.408863. Entropy: 0.323500.
Iteration 1590: Policy loss: -0.002275. Value loss: 6.129225. Entropy: 0.308942.
episode: 656   score: 105.0  epsilon: 1.0    steps: 368  evaluation reward: 186.7
episode: 657   score: 155.0  epsilon: 1.0    steps: 400  evaluation reward: 186.45
episode: 658   score: 485.0  epsilon: 1.0    steps: 984  evaluation reward: 189.2
Training network. lr: 0.000238. clip: 0.095245
Iteration 1591: Policy loss: 0.000933. Value loss: 31.185831. Entropy: 0.429799.
Iteration 1592: Policy loss: -0.006547. Value loss: 19.621401. Entropy: 0.475382.
Iteration 1593: Policy loss: -0.002770. Value loss: 18.094185. Entropy: 0.449148.
episode: